In [1]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

In [2]:
load_dotenv(find_dotenv())  # load .env file

True

In [3]:
client = OpenAI()

In [4]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Hello!"},
    ],
)

In [5]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [6]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": self.system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        response = client.chat.completions.create(
            model="gpt-4-0125-preview",
            temperature=0,
            messages=self.messages,
        )
        return response.choices[0].message.content

In [7]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observations will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if neccessary.

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed.

Example session:

Question: How much does a bulldog weigh ?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A bulldog weighs 50 pounds.

You then output:

Answer: A bulldog weighs 50 pounds.
""".strip()

In [8]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terrier average 20 pounds")
    elif name in "Border Collie":
        return("Border Collie average 37 pounds")
    elif name in "Toy Poodle":
        return("Toy Poodle average 50 pounds")
    else:
        return(f"An average dog weigh 50 pounds")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight,
}

In [9]:
abot = Agent(prompt)

In [10]:
result = abot("How much does a Scottish Terrier weigh ?")
print(result)

Thought: I should look up the dog's weight using average_dog_weight for a Scottish Terrier.
Action: average_dog_weight: Scottish Terrier
PAUSE


In [11]:
result = average_dog_weight("Scottish Terrier")
result

'Scottish Terrier average 20 pounds'

In [12]:
next_prompt = f"Observation: {result}"

In [13]:
abot(next_prompt)

'Answer: A Scottish Terrier weighs an average of 20 pounds.'

In [14]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservations will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if neccessary.\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed.\n\nExample session:\n\nQuestion: How much does a bulldog weigh ?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A bulldog weighs 50 pounds.\n\nYou then output:\n\nAnswer: A bulldog weighs 50 pounds.'},
 {'role': 'user', 'content': 'How m

In [15]:
abot = Agent(prompt)

In [16]:
question = """
I have 2 dogs, a border collie and a toy poodle. What is their combined weight ?
"""
print(abot(question))

Thought: To find the combined weight of a border collie and a toy poodle, I need to find the average weight of each breed and then add them together.
Action: average_dog_weight: Border Collie
PAUSE


In [17]:
next_prompt = f"Observation: {average_dog_weight('Border Collie')}"
abot(next_prompt)

'Thought: Now that I know the average weight of a Border Collie, I need to find the average weight of a Toy Poodle to calculate the combined weight.\nAction: average_dog_weight: Toy Poodle\nPAUSE'

In [18]:
next_prompt = f"Observation: {average_dog_weight('Toy Poodle')}"
abot(next_prompt)

"Thought: The observation about the Toy Poodle's average weight seems incorrect based on my knowledge; Toy Poodles are much smaller. However, I will proceed with the given information to calculate the combined weight.\nAction: calculate 37 + 50\nPAUSE"

In [19]:
next_prompt = f"Observation: {calculate('37 + 50')}"
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Toy Poodle, based on the provided average weights, is 87 pounds.'

## Automated

In [20]:
action_re = re.compile("^Action: (\w+): (.*)$")

In [21]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(line)
            for line in result.split("\n")
            if action_re.match(line)
        ]

        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action {action}")
            print(f"--- running {action} {action_input}")
            observation = known_actions[action](action_input)
            print(f"Observation {observation}")
            next_prompt = f"Observation: {observation}"
        else:
            return

In [25]:
question = "I have 2 dogs, a border collie and a acottish terrier. What is their combined weight ?"
query(question)

Thought: To find the combined weight of the two dogs, I need to find the average weight of a Border Collie and a Scottish Terrier separately and then add those weights together.

Action: average_dog_weight: Border Collie
PAUSE
--- running average_dog_weight Border Collie
Observation Border Collie average 37 pounds
Thought: Now that I know the average weight of a Border Collie is 37 pounds, I need to find the average weight of a Scottish Terrier to calculate the combined weight.

Action: average_dog_weight: Scottish Terrier
PAUSE
--- running average_dog_weight Scottish Terrier
Observation Scottish Terrier average 20 pounds
Thought: With the average weight of a Border Collie being 37 pounds and a Scottish Terrier being 20 pounds, I can now calculate their combined weight.

Action: calculate 37 + 20
PAUSE
